In [ ]:
# Cell 1: Setup & Path Fix
import sys
import os
from pathlib import Path

current_dir = Path.cwd()
root_dir = current_dir if (current_dir / "src").exists() else current_dir.parent

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

print(f"Project Root set to: {root_dir}")

from src import config, preprocessing, rf_classifier, evaluation, utils
from sklearn.model_selection import train_test_split
import numpy as np
import gc

exp_paths = utils.setup_experiment_folder()
print(f"Baseline Experiment Initialized.")

✅ Project Root set to: c:\Users\Admin\Documents\ids_ae_rf_hybrid
📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_194815
🚀 Baseline Experiment Initialized.


In [ ]:
print("Loading and Splitting data (2017 & 2018)...")

# 1. Load data
X_17, y_17 = preprocessing.load_single_dataset_year('2017', binary_mode=True)
X_18, y_18 = preprocessing.load_single_dataset_year('2018', binary_mode=True)

# 2. Split 80/20 for each year
from sklearn.model_selection import train_test_split
X_17_tr, X_17_te, y_17_tr, y_17_te = train_test_split(X_17, y_17, test_size=0.2, random_state=config.SEED, stratify=y_17)
X_18_tr, X_18_te, y_18_tr, y_18_te = train_test_split(X_18, y_18, test_size=0.2, random_state=config.SEED, stratify=y_18)

# 3. Mix training data from both years
X_train_mix = np.vstack([X_17_tr, X_18_tr])
y_train_mix = np.concatenate([y_17_tr, y_18_tr])

print(f"Mixed Train Size: {X_train_mix.shape}")

# RAM cleanup
del X_17, X_18, X_17_tr, X_18_tr
gc.collect()

🚀 Loading and Splitting data (2017 & 2018)...
🔄 Loading dataset year 2017 (Binary=True)...
✅ Loaded 2017. Shape: (2830743, 65)
🔄 Loading dataset year 2018 (Binary=True)...
✅ Loaded 2018. Shape: (9625148, 65)
✅ Mixed Train Size: (9964712, 65)


10

In [ ]:
# 1. Scaling based on Train Mix
scaler = preprocessing.get_scaler()
X_train_scaled = scaler.fit_transform(X_train_mix)

# 2. Get Index of Top 30 mRMR features
# (config.mRMR_FEATURES contains list of feature names sorted by mRMR)
K = 30 
top_mrmr_features = config.mRMR_FEATURES[:K]
mrmr_indices = [config.SELECTED_FEATURES.index(f) for f in top_mrmr_features]

# 3. Keep only mRMR features
X_train_final = X_train_scaled[:, mrmr_indices]

print(f"Final Training Shape (mRMR only): {X_train_final.shape}")

import joblib
joblib.dump(scaler, exp_paths["models"] / "baseline_scaler.joblib")

✅ Final Training Shape (mRMR only): (9964712, 20)


['C:\\Users\\Admin\\Documents\\ids_ae_rf_hybrid\\results\\experiments\\exp_20251221_194815\\models\\baseline_scaler.joblib']

In [ ]:
print("Training Baseline Random Forest...")

rf_save_path = exp_paths["models"] / "rf_baseline_model.joblib"

# Train the model
rf_model = rf_classifier.train_rf(X_train_final, y_train_mix, save_path=rf_save_path)

print("Baseline RF training completed.")

🚀 Training Baseline Random Forest...
[RandomForest] Training classifier...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:  2.3min


building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75

[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:  7.3min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251221_194815\models\rf_baseline_model.joblib
✅ Baseline RF training completed.


In [ ]:
print("\n--- STARTING FINAL EVALUATION (BASELINE) ---")

# Prepare test sets (Scale and select features)
def prepare_test_set(X_test_raw):
    X_sc = scaler.transform(X_test_raw)
    return X_sc[:, mrmr_indices]

test_scenarios = [
    (prepare_test_set(X_17_te), y_17_te, "Baseline - Unseen 2017"),
    (prepare_test_set(X_18_te), y_18_te, "Baseline - Unseen 2018"),
    (prepare_test_set(np.vstack([X_17_te, X_18_te])), 
     np.concatenate([y_17_te, y_18_te]), "Baseline - Global Mixed Test")
]

# Save results for later comparison
baseline_results = {}

for X_t, y_t, name in test_scenarios:
    print(f"\nScenario: {name}")
    metrics = evaluation.evaluate_model(
        model=rf_model, 
        X_test=X_t, 
        y_test=y_t, 
        save_dir=exp_paths["figures"], 
        dataset_name=name
    )
    baseline_results[name] = metrics

print("\nAll evaluations done.")


--- STARTING FINAL EVALUATION (BASELINE) ---


c:\Users\Admin\miniforge3\envs\ids-thesis\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Users\Admin\miniforge3\envs\ids-thesis\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(



📊 Scenario: Baseline - Unseen 2017

📊 Evaluating on Baseline - Unseen 2017...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.5s finished


   ✅ Accuracy: 0.9911
   ⭐ MCC:      0.9723
   📝 Report saved to: report_Baseline_-_Unseen_2017.txt
   🖼️ Confusion Matrix saved to: cm_Baseline_-_Unseen_2017.png

📊 Scenario: Baseline - Unseen 2018

📊 Evaluating on Baseline - Unseen 2018...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    2.2s finished


   ✅ Accuracy: 0.9726
   ⭐ MCC:      0.9329
   📝 Report saved to: report_Baseline_-_Unseen_2018.txt
   🖼️ Confusion Matrix saved to: cm_Baseline_-_Unseen_2018.png

📊 Scenario: Baseline - Global Mixed Test

📊 Evaluating on Baseline - Global Mixed Test...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    2.3s finished


   ✅ Accuracy: 0.9768
   ⭐ MCC:      0.9406
   📝 Report saved to: report_Baseline_-_Global_Mixed_Test.txt
   🖼️ Confusion Matrix saved to: cm_Baseline_-_Global_Mixed_Test.png

✅ All evaluations done.
